In [ ]:
! pip install pywikibot
! pip install mwparserfromhell
! pip install watchdog

In [1]:
import os

os.environ['PYWIKIBOT_DIR'] = './wiki_reader/'

In [ ]:
# import sparknlp
# from pyspark.sql import SQLContext

# spark = sparknlp.start()
# sc = spark.sparkContext
# sqlCtx = SQLContext(sc)

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType, DoubleType

from spark_app.scorers import score_text
from spark_app.spark_tools import SparkSentimentStreamer
from pathlib import Path

## Input

In [3]:
request = 'peano'

batch_size = 100
limit = None
preload_content = True
is_category = False

## Processing

In [ ]:
# from ml.LogisticRegressionCached import readFromCache

# (lrModel, pipelineModel) = readFromCache('./ml/train/')

# def score_text_ml(text):
#     df = spark.createDataFrame([(text, 2)], ['text', 'target'])
#     df_transformed = pipelineModel.transform(df) # To fix
#     predictions = lrModel.transform(df_transformed)
#     predictions = predictions.select(['text', 'probability', 'prediction'])
#     pd_predictions = predictions.toPandas()
#     positive_probability = pd_predictions.iloc[0]['probability'][1]
#     overall_probability = 2 * positive_probability - 1
    
#     return overall_probability

In [4]:
from ml.OutputProcessing import process_output_files

#process_output_files('./responses/', 1000)

IntProgress(value=0, description='Iter 0/1000', max=1000)

'Mean: 0.615'

,name,neg,neu,pos,compound
0,Page #0,0.199,0.856,0.750,0.833
1,Page #1,0.005,0.380,0.906,0.397


In [ ]:
def spark_process(request, score_func):
    spark = SparkSession.builder\
        .master("local[*]")\
        .appName("NetworkWordCount")\
        .getOrCreate()
    
    sc = spark.sparkContext
    ssc = StreamingContext(sc, 1)
    
    dataInp = "requests/" + request
    dataOut = "responses/" + request
    Path(dataOut).mkdir(parents=True, exist_ok=True)
    
    streamer = SparkSentimentStreamer(sc, ssc, spark, score_func, dataInp, dataOut)
    streamer.run()
    streamer.stop()

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import wiki_reader.reader as reader

wrapper = lambda r,b,l,cat,cont: reader.query(r,batch_size=b,limit=l,is_category=cat,preload_content=cont)

with ThreadPoolExecutor(max_workers=2) as e:
    e.submit(wrapper, 'peano', batch_size, limit, is_category, preload_content)
    e.submit(spark_process, 'peano', score_text)
    # e.submit(process_output_files, './responses/', 1000)